# Exercício 07_01

#### Juliana Carolina Thuler dos Santos

## Árvore

Criar o código necessários para que os métodos `.cria_regras()` e `.predicao()` cumpram seus objetivos. O primeiro deve percorrer a árvore já criada pelo `.cria_galhos()` e armazeanda nos atributos `filhos`, armazenando em um dicionário as regras para cada quebra. Já o segundo deve para uma nova instância, percorrer as regras criadas pelo método anterior e fazer a predição baseado nisso.

Você pode passar nos parâmetros para os métodos que julgar necessário.

Lembrando que partimos das premissas que:

- Nosso problema terá como variáveis explicativas apenas dados categóricos;
- Nosso problema terá como variável resposta apenas dados categóricos;

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import mode

df = pd.read_csv('titanic.csv')

df_amostra = df.sample(15, random_state = 1)

In [2]:
class Arvore:
    
    def __init__(self, 
                 df, var_resposta, 
                 atributo_origem = '-',
                 atributo_origem_valor = '-', 
                 profundidade = 0):
        
        self.df = df
        self.var_resposta = var_resposta
        self.filhos = []
        self.atributo_melhor = '-'
        self.atributo_origem = atributo_origem
        self.atributo_origem_valor = atributo_origem_valor
        self.gi = 0
        self.profundidade = profundidade
        self.regras = []
    
    def __repr__(self):
        return f'''Nó {self.profundidade} da Árvore de Decisão.\nCaracterísticas do No:
            Dataframe de {self.df.shape[0]} linhas.
            A origem desse nó é o atributo {self.atributo_origem} com valor {self.atributo_origem_valor}
            Melhor atributo desse nó é {self.atributo_melhor} com GI de {self.gi}'''
    
    @staticmethod
    def entropia_binaria(p):
    
        if p == 0 or p == 1:
            return 0

        n = 1 - p
        return -(p*np.log2(p) + n*np.log2(n))
    
    @staticmethod
    def ganho_informacao(df, coluna, var_resposta):
    
        seq_poss = df[coluna].unique()

        ent_inicial = Arvore.entropia_binaria(df[var_resposta].value_counts(1).iloc[0])

        conj = [df[df[coluna] == i] for i in seq_poss]
        ent = [Arvore.entropia_binaria(i[var_resposta].value_counts(1).iloc[0]) for i in conj]
        n_elem = [i.shape[0] for i in conj]

        total = sum(n_elem)

        ent_atr = 0
        for i in range(len(conj)):
            ent_atr += ent[i]*n_elem[i]/total

        return ent_inicial - ent_atr
        
    def melhor_atributo(self):
        '''
        Esse método acha o melhor atributo em relação ao ganho de informação.
        '''

        colunas_a_testar = self.df.columns.to_list()
        colunas_a_testar.remove(self.var_resposta)

        maior_v = 0
        maior_k = '-'

        for col in colunas_a_testar:

            v = Arvore.ganho_informacao(self.df, col, self.var_resposta)
            if v > maior_v:
                maior_v = v
                maior_k = col
        
        return maior_k, maior_v
        
    def realiza_quebra(self):
        '''
        Quebra o conjunto de dados inicial em N subconjuntos de acordo com 
        o melhor atributo.
        '''
        
        self.atributo_melhor, self.gi = self.melhor_atributo()
        
        if self.atributo_melhor == '-':
            return 
        
        possibilidades = self.df[self.atributo_melhor].unique()
        
        for p in possibilidades:
            mascara = self.df[self.atributo_melhor] == p
            sub_conj = self.df.loc[mascara, :]
            self.filhos.append(Arvore(sub_conj, 
                                      self.var_resposta, 
                                      atributo_origem = self.atributo_melhor,
                                      atributo_origem_valor = p,
                                      profundidade = self.profundidade + 1))
            
            
    def cria_galhos(self):
        
        self.realiza_quebra()
        
        for i in self.filhos:
            
            p0 = i.df[i.var_resposta].value_counts(1).iloc[0]
            ent = Arvore.entropia_binaria(p0)
            
            if ent > 0:
                i.cria_galhos()
                
    
    # TODO
    def cria_regras(self):
        '''
        Esse método deve percorrer recurssivamente as árvores retornando o
        conjunto de regras que definem a qual nó a nova instância pertencerá.
        '''
        
        self.regras = [self.atributo_origem, self.atributo_origem_valor, self.atributo_melhor,self.gi]
        
        # Condição de parada:
        if self.regras[3] == 0:
            pass
        
        else:
            for i in self.filhos:
                i.cria_regras()
                
        return self.regras
    
    # TODO
    def predicao(self, nova_instancia):
        ''' 
        Dado uma nova instância, retorna a sua predição.
        '''
        
        df = self.df[self.df[self.regras[2]] == nova_instancia[self.regras[2]]]
        nova_instancia = nova_instancia.drop(labels = [self.regras[2]])
        
        if len(nova_instancia) == 0:
            pass
        
        else:
        
            for i in self.filhos:
                        
                for j in nova_instancia:
                    if j == i.regras[1]:
                        i.predicao(nova_instancia)
            
        lista = list(df['Survived'])
        pred = mode(lista).mode[0]                
        
        return pred

In [3]:
df_amostra = df_amostra[['Survived', 'Parch', 'Sex', 'SibSp', 'Pclass']]

x_teste = df_amostra.drop(columns = ['Survived'])
y_teste = df_amostra[['Survived']]

In [4]:
arv = Arvore(df_amostra[['Survived', 'Parch', 'Sex', 'SibSp', 'Pclass']], 'Survived')

In [5]:
arv.cria_galhos()

In [6]:
#cria_regras()

In [7]:
arv.cria_regras()

['-', '-', 'Sex', 0.430776632270099]

In [8]:
#veio de: 'Sex'
#'Sex' = female' / 'male'
#próxima quebra: 'Pclass' / 'Pclass'
#GI das próximos quebras

for i in range(len(df['Sex'].unique())):
    print(arv.filhos[i].regras)

['Sex', 'female', 'Pclass', 0.13792538097003]
['Sex', 'male', 'Pclass', 0.3059584928680418]


In [9]:
#predicao()

In [10]:
predict = []
for linha in range(len(x_teste)):
    p = arv.predicao(x_teste.iloc[linha,:])
    predict.append(p)

In [13]:
from sklearn.metrics import accuracy_score
round(accuracy_score(predict,y_teste),3)

0.867

In [12]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_teste, predict).ravel()
print(tn, fp, fn, tp)

6 1 1 7
